## Inteligență Artificială - Tema 1
 - Alexandru-Marian Belbu

#### Functia de citire a json-ului

In [1]:
import json
import os



def read_json(test_name):
    path = os.path.join("./tests", test_name + ".json")
    d = json.load(open(path))

    total_black = 0
    for i in range(d["height"]):
        constraint = d["rows"][i]
        for value in constraint:
            total_black = total_black + value
    print("Numarul total de celule negre:",total_black)

    d["total_black"] = total_black

    return d


game = read_json("test1")
print(game)



Numarul total de celule negre: 13
{'name': 'smile', 'height': 5, 'width': 5, 'rows': [[2, 2], [2, 2], [], [1, 1], [3]], 'columns': [[2, 1], [2, 1], [1], [2, 1], [2, 1]], 'total_black': 13}


#### Crearea unei functii care sa creeze nodul root

Un nod va fi de tipul tabel | nr_celule_negre  (acest lucru ajuta mai tarziu pentru a nu verifica de fiecare data daca un nod este valid sau nu)

In [2]:
def create_root(height = game["height"], width = game["width"]):
    return ([[False for _ in range(width)] for _ in range(height)], 0)

print(create_root())

([[False, False, False, False, False], [False, False, False, False, False], [False, False, False, False, False], [False, False, False, False, False], [False, False, False, False, False]], 0)


#### Generarea vecinilor
Vecinul unui nod Node(Table, Crt_Black) sunt toate nodurile care au numarul de celule negre (Crt_Black + 1)
Cu toate acestea, pentru a pastra un numar relativ acceptabil de vecini verific doar daca depasesc numarul maxim de celule negre pe linii conform constrangerilor

In [3]:
import copy

def generate_neighbours(table):
    new_tables = []
    for i in range(game["height"]):
        constraint = game["rows"][i]
        line_black = sum(constraint)
        nr_black = sum(table[i])
        if nr_black >= line_black:
            continue
        for j in range(game["width"]):
            if not table[i][j]:
                new_table = copy.deepcopy(table)
                new_table[i][j] = True
                new_tables.append(new_table)
    return new_tables


def get_neighbours(node):
    (table, nr_black) = node
    if nr_black == game["total_black"]:
        return []
    new_tables = generate_neighbours(table)
    rez = list(map(lambda x: (x, nr_black + 1), new_tables))
    return rez


neighbours = get_neighbours(create_root())
print("Vecinul 1 al lui root:")
for row in neighbours[0][0]:
    print(row)
print("\n")
print("Vecinul 2 al lui root:")
for row in neighbours[1][0]:
    print(row)
print("...................................\n")

Vecinul 1 al lui root:
[True, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]


Vecinul 2 al lui root:
[False, True, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
[False, False, False, False, False]
...................................



#### Verificarea solutiei

Pentru a valida o solutie pentru aceasta problema este nevoie de verificare constrangerilor pe toate liniile, respectiv toate coloanele

In [6]:
def check_valid_cols(table):
    for col in range(game["width"]):
        constraints = game["columns"][col]
        i = 0
        while i < game["height"]:
            if not constraints:
                while i < game["height"]:
                    if table[i][col]:
                        return False
                    i = i + 1
                break
            if table[i][col]:
                nr_blocks = 0
                while i < game["height"] and table[i][col]:
                    nr_blocks = nr_blocks + 1
                    i = i + 1
                if nr_blocks == constraints[0]:
                    constraints = constraints[1:]
                    continue
                else:
                    return False
            else:
                while i < game["height"] and not table[i][col]:
                    i = i + 1
        if constraints:
            return False
    return True

def check_valid_lines(table):
    for l in range(game["height"]):
        constraints = game["rows"][l]
        i = 0
        while i < game["width"]:
            if not constraints:
                while i < game["width"]:
                    if table[l][i]:
                        return False
                    i = i + 1
                break
            if table[l][i]:
                nr_blocks = 0
                while i < game["width"] and table[l][i]:
                    nr_blocks = nr_blocks + 1
                    i = i + 1
                if nr_blocks == constraints[0]:
                    constraints = constraints[1:]
                    continue
                else:
                    return False
            else:
                while i < game["width"] and not table[l][i]:
                    i = i + 1
        if constraints:
            return False
    return True


def check_solution(node):
    (table, nr_black) = node
    if nr_black != game["total_black"]:
        return False
    return check_valid_lines(table) and check_valid_cols(table)


# Aceasta este solutia unica pentru testul 1 (smile)
true_solution = ([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)
print("Check true solution:", check_solution(true_solution))

print("Check empty node:", check_solution(create_root()))


Check true solution: True
Check empty node: False


#### Implementare pentru id-ul unui nod (este nevoie pentru ca un tuplu tabel - int nu are un hashcode pentru a putea fi pus intr-un dictionar)

In [7]:
def get_id(node):
    hashc = ""
    for i in range(game["height"]):
        for j in range(game["width"]):
            if node[0][i][j]:
                hashc = hashc + 'a'
            else:
                hashc = hashc + 'b'
    return hashc

print("ID empty node:    ", get_id(create_root())) 
print("ID true solution: ", get_id(true_solution))

ID empty node:     bbbbbbbbbbbbbbbbbbbbbbbbb
ID true solution:  aabaaaabaabbbbbabbbabaaab


#### Implementarea BFS-ului

Dezavantajele acestei implementari este faptul ca trebuie create aproape tot arborele de "solutii intermediare" pentru a ajunge la solutia problemei
Acest lucru duce la folosirea memoriei intr-un mod ineficient cat si la un timp de executie indelungat

In [8]:
def solve_bfs():
    queue = [create_root()]
    visited = {}
    while queue:
        node = queue[0]
        queue.pop(0)
        visited[get_id(node)] = True
        neighbours = get_neighbours(node)
        for neighbour in neighbours:
            if check_solution(neighbour):
                return neighbour
            else:
                id_neighbour = get_id(neighbour)
                if id_neighbour not in visited:
                    visited[id_neighbour] = True
                    queue.append(neighbour)
    return None


print(solve_bfs())

([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)


#### Implementare DFS

Aceasta solutie se dovedeste a fi mai utila decat bfs pe motiv ca pe acest test solutia dupa cum au fost creati vecinii se afla in prima parte a arborelui (partea stanga a arborelui mai exact)
Implementarea este eficienta dpdv al memoriei pe motiv ca se vor pastra in memorie aproximativ (depth * tabele) zone de memorie (mai mult pentru ca se creaza si vecinii fiecarui nod de pe "inaltimea arborelui" )

In [9]:
def dfs(node, visited):
    if(check_solution(node)):
        return node
    neighbours = get_neighbours(node)
    for neighbour in neighbours:
        neighbour_id = get_id(neighbour)
        if neighbour_id in visited:
            continue
        visited[neighbour_id] = True 
        rez = dfs(neighbour, visited)
        if rez:
            return rez
    return None


def solve_dfs():
    root = create_root()
    visited = {}
    return dfs(root, visited)

print(solve_dfs())

([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)


#### Implementare Iterative Deepening

Se poate observa ca aceasta implementare este cea mai putin avantajoasa (cand sti exact adancimea la care se afla solutia)
Se vede cu ochiul liber ca este mult mai dezavantajoasa si decat bfs (ceea ce este si normal) pentru ca va cauta cam in acelasi numar de noduri ca si bfs-ul (si chiar mai multi pentru ca trebuie repeata functia cu adancime din ce in ce mai mare deci cu cat un nod este "mai sus" in arbore (mai aproape de root) cu atat mai multe vizitari ale implementarii va avea)

In [10]:
def dfs_iterative(node, depth, visited):

    if depth == 0:
        return None
    depth = depth - 1 
    if(check_solution(node)):
        return node
    neighbours = get_neighbours(node)
    for neighbour in neighbours:
        neighbour_id = get_id(neighbour)
        if neighbour_id in visited:
            continue
        visited[neighbour_id] = True 
        rez = dfs_iterative(neighbour, depth, visited)
        if rez:
            return rez
    return None


def solve_iterative():
    root = create_root()
    depth = 1
    while True:
        visited = {}
        sol = dfs_iterative(root, depth, visited)
        if sol:
            return sol
        depth = depth + 1

print(solve_iterative())

    

([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)


#### Crearea functiilor euristice:

Am decis ca functia euristica sa se bazeze pe numarul de grupuri corecte si posibilitatea continuarii solutiei (odata nr de grupuri de pe linii, nr de grupuri pe coloana, si apoi combinat)

In [11]:
INFINITE = 100000000


def get_col_score(table, col ,constraint):
    i = 0
    score = 0.0

    nr_constraint = len(constraint)

    while i < game["height"]:
        if table[i][col]:
            if not constraint:
                return INFINITE
            else:
                nr_black = 0
                while i < game["height"] and table[i][col]:
                    nr_black = nr_black + 1
                    i = i + 1
                if nr_black == constraint[0]:
                    score = score + float(constraint[0])
                    constraint = constraint[1:]
                    if i == game["height"] and constraint:
                        return INFINITE
                    continue
                elif nr_black < constraint[0]:
                    needed_space = constraint[0] - nr_black + len(constraint[1:]) + sum(constraint[1:])
                    while i < game["height"] and not table[i][col]:
                        needed_space = needed_space - 1
                        i = i + 1
                    if needed_space <= 0 and i == game["height"]:
                        return score + float(nr_black)
                    elif needed_space > 0 or i == game["height"]:
                        return INFINITE
                else:
                    return INFINITE
        else:
            nr_white = 0
            while i < game["height"] and not table[i][col]:
                i = i + 1
                nr_white = nr_white + 1
            if i == game["height"] and constraint:
                needed_space = len(constraint) + sum(constraint)
                if len(constraint) == nr_constraint:
                    needed_space = needed_space - 1
                if needed_space > nr_white:
                    return INFINITE
            if i == game["height"] and not constraint:
                return score
    return score


def column_heuristic(node):
    (table, _) = node
    constraints = game["columns"]
    total_blocks = 0
    for constraint in constraints:
        total_blocks = total_blocks + sum(constraint)

    corect_groups = 1.0

    for col_index in range(game["width"]):
        c = game["columns"][col_index]

        score = get_col_score(table, col_index, c)

        if score == INFINITE:
            return INFINITE

        corect_groups = corect_groups + score

    return (float(total_blocks) - float(node[1])) / float(corect_groups)

def get_line_score(table, line, constraint):
    j = 0
    score = 0.0

    nr_constraint = len(constraint)

    while j < game["width"]:
        if table[line][j]:
            if not constraint:
                return INFINITE
            else:
                nr_black = 0
                while j < game["width"] and table[line][j]:
                    nr_black = nr_black + 1
                    j = j + 1
                if nr_black == constraint[0]:
                    score = score + float(constraint[0])
                    constraint = constraint[1:]
                    if j == game["width"] and constraint:
                        return INFINITE
                    continue
                elif nr_black < constraint[0]:
                    needed_space = constraint[0] - nr_black + len(constraint[1:]) + sum(constraint[1:])
                    while j < game["width"] and not table[line][j]:
                        needed_space = needed_space - 1
                        j = j + 1
                    if needed_space <= 0 and j == game["width"]:
                        return score + float(nr_black)
                    elif needed_space > 0 or j == game["width"]:
                        return INFINITE
                else:
                    return INFINITE
        else:
            nr_white = 0
            while j < game["width"] and not table[line][j]:
                j = j + 1
                nr_white = nr_white + 1
            if j == game["width"] and constraint:
                needed_space = len(constraint) + sum(constraint)
                if len(constraint) == nr_constraint:
                    needed_space = needed_space - 1
                if needed_space > nr_white:
                    return INFINITE
            if j == game["width"] and not constraint:
                return score
    return score

def line_heuristic(node):
    (table, _) = node
    constraints = game["rows"]
    total_blocks = 0
    for constraint in constraints:
        total_blocks = total_blocks + sum(constraint)

    corect_groups = 1.0

    for row_index in range(game["height"]):
        c = game["rows"][row_index]

        score = get_line_score(table, row_index, c)

        if score == INFINITE:
            return INFINITE

        corect_groups = corect_groups + score
    return (float(total_blocks) - float(node[1])) / float(corect_groups)

def col_and_line_heuristic(node):
    (table, _) = node
    constraints = game["rows"]
    total_blocks = 0
    for constraint in constraints:
        total_blocks = total_blocks + sum(constraint)

    corect_groups = 1.0

    for row_index in range(game["height"]):
        c = game["rows"][row_index]

        score = get_line_score(table, row_index, c)

        if score == INFINITE:
            return INFINITE

        corect_groups = corect_groups + score
    
    for col_index in range(game["width"]):
        c = game["columns"][col_index]

        score = get_col_score(table, col_index, c)

        if score == INFINITE:
            return INFINITE

        corect_groups = corect_groups + score

    return (float(total_blocks) - float(node[1])) / float(corect_groups)

    

#### Implementare A*

In [12]:
from heapq import heappush, heappop

def aStar(h):
    frontier = []
    root = create_root()
    heappush(frontier, (h(root), root))
    discovered = {get_id(root): (None, 0)}
    while frontier:
        (_, node) = heappop(frontier)
        cost = discovered[get_id(node)][1]
        (table, crt_black) = node
        if crt_black == game["total_black"] and check_solution(node):
            return node
        neighbours = get_neighbours(node)
        for neighbour in neighbours:
            cost_new = 1.0 + cost
            neigh_id = get_id(neighbour)
            if neigh_id in discovered:
                continue
            else:
                discovered[neigh_id] = (node, cost_new)
                heappush(frontier, (cost_new + h(neighbour), neighbour))
    return None


In [13]:
print(aStar(line_heuristic))

([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)


In [14]:
print(aStar(column_heuristic))

([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)


In [15]:
print(aStar(col_and_line_heuristic))

([[True, True, False, True, True], [True, True, False, True, True], [False, False, False, False, False], [True, False, False, False, True], [False, True, True, True, False]], 13)


Se poate vedea cu ochiul liber ca aStar in acest caz este superior metodelor clasice de cautare (dfs, bfs)
- Cea mai proasta heuristica s-a dovedit a fi column_heuristic (acest lucru se poate datora si din faptul ca pentru a calcula trebuiesc iterate coloanele si asta nu prea ajuta la nivelul de cache)
- A doua cea mai buna este line_heuristic (o parte se datoreaza si datorita iterarii pe linii deci datele sunt asezate mai bine in cache)
- Cea mai buna varianta s-a dovedit a fi o combinatie a celor doua (chiar daca vom itera si coloanele ne ajuta mult sa "arunca" la finalul priority queue-ului solutiile care nu respecta constrainturile jocului)

#### Grafice
Din pacate nu am putut realiza grafice pe motiv ca doar primele doua teste functioneaza pe implementarea mea (avand in vedere ca fiecare nod are un numar foarte mare de vecini deci si complexitatea e pe masura (si spatiala -> ajungand la pe putin 12 gb ram si temporala -> durand mai mult de 15 minute))

#### Implementarea MAC

Voi reimplementa functiile de baza (get_neighbours, create_root, etc) pentru a se putea folosi mai usor pentru implementarea de mac
Ma voi baza pe crearea tabelelor generand toate posibilitatile unei linii (generare corecta luand in verdere constrangerile)
Deci rootul va fi o tabela goala (fara nicio linie) si numarul de linii adica 0

Pe parcursul programului voi completa nodul curent cu toate posibilitatile urmatoarei linii

Deci:
create_root_mac -> va intoarce o tabla goala si 0

generate_rows -> va genera toate posibilitatile de linie avand ca parametri constrangerile

check_solution_mac -> este nevoie de o noua functie de check_solution pe motiv ca va trebui verificat doar coloanele (nu si liniile pentru ca stiu sigur ca sunt corect fiind generate de mine)


In [17]:
def create_root_mac():
    return ([], 0)


def generate_rows(constraints, width):
    nr_constraints = len(constraints)
    if nr_constraints == 0:
        return [[False] * width]
    rows = []
    min_needed_spaces = sum(constraints) + len(constraints) - 1
    trailing_white = 0
    while trailing_white + min_needed_spaces <= width:
        new_width = width - trailing_white - constraints[0]
        if nr_constraints > 1:
            new_width = new_width - 1
        right_parts = generate_rows(constraints[1:], new_width)
        left_part = trailing_white * [False] + constraints[0] * [True]
        if nr_constraints > 1:
            left_part = left_part + [False]
        combinations = list(map(lambda x: copy.copy(left_part) + x, right_parts))
        rows = rows + combinations
        trailing_white = trailing_white + 1
    return rows

def check_solution_mac(node, my_game):
    if not node[1] == my_game["height"]:
        return False
    table = node[0]
    for col in range(my_game["width"]):
        constraints = my_game["columns"][col]
        i = 0
        while i < my_game["height"]:
            if not constraints:
                while i < my_game["height"]:
                    if table[i][col]:
                        return False
                    i = i + 1
                break
            if table[i][col]:
                nr_blocks = 0
                while i < my_game["height"] and table[i][col]:
                    nr_blocks = nr_blocks + 1
                    i = i + 1
                if nr_blocks == constraints[0]:
                    constraints = constraints[1:]
                    continue
                else:
                    return False
            else:
                while i < my_game["height"] and not table[i][col]:
                    i = i + 1
        if constraints:
            return False
    return True

def check_intermediate(node, my_game):
    if node[1] == my_game["total_black"]:
        return check_solution_mac(node, my_game)
    table = node[0]
    nr_lines = node[1]
    for col in range(my_game["width"]):
        constraints = my_game["columns"][col]
        i = 0
        while i < nr_lines:
            if not constraints:
                while i < nr_lines:
                    if table[i][col]:
                        return False
                    i = i + 1
                break
            if table[i][col]:
                nr_blocks = 0
                while i < nr_lines and table[i][col]:
                    nr_blocks = nr_blocks + 1
                    i = i + 1
                if nr_blocks == constraints[0]:
                    constraints = constraints[1:]
                    continue
                elif nr_blocks > constraints[0]:
                    return False
                else:
                    while i < nr_lines:
                        if table[i][col]:
                            return False
                        i = i + 1
                    break
            else:
                while i < nr_lines and not table[i][col]:
                    i = i + 1
    return True


def get_neighbours_mac(node, my_game):
    if node[1] == my_game["height"]:
        return []
    table = node[0]
    next_row = node[1]
    neighbours = []
    new_rows = generate_rows(my_game["rows"][next_row], my_game["width"])
    for i in range(len(new_rows)):
        new_neighbour = (copy.deepcopy(node[0]), node[1] + 1)
        new_neighbour[0].append(new_rows[i])
        neighbours.append(new_neighbour)
    return list(filter(lambda x: check_intermediate(x, my_game), neighbours))   #aici ma asigur de arc-consistenta

Implementarea functiei mac pt gasirea solutiei

In [18]:
def solve_mac(test_name):
    my_game = read_json(test_name)
    queue = [create_root_mac()]
    print(create_root_mac())
    while queue:
        node = queue[0]
        queue.pop(0)
        neighbours = get_neighbours_mac(node, my_game)
        for neighbour in neighbours:
            if check_solution_mac(neighbour, my_game):
                return neighbour
            else:
                queue.append(neighbour)
    return None

sol = solve_mac("test1")
for row in sol[0]:
    print(row)

Numarul total de celule negre: 13
([], 0)
[True, True, False, True, True]
[True, True, False, True, True]
[False, False, False, False, False]
[True, False, False, False, True]
[False, True, True, True, False]


In [19]:
sol = solve_mac("test2")
for row in sol[0]:
    print(row)

Numarul total de celule negre: 16
([], 0)
[True, False, True, False, True]
[True, True, True, True, True]
[False, True, True, True, False]
[False, True, False, True, False]
[False, True, True, True, False]


In [20]:
sol = solve_mac("test3")
for row in sol[0]:
    print(row)

Numarul total de celule negre: 46
([], 0)
[False, False, False, False, False, False, True, True, True, True]
[False, False, False, True, True, True, False, False, False, True]
[False, False, False, True, False, False, False, True, True, True]
[False, False, False, True, True, True, True, False, False, True]
[False, False, False, True, False, False, False, False, False, True]
[False, False, False, True, False, False, False, True, True, True]
[False, True, True, True, False, False, True, True, True, True]
[True, True, True, True, False, False, True, True, True, True]
[True, True, True, True, False, False, False, True, True, False]
[False, True, True, False, False, False, False, False, False, False]


In [21]:
sol = solve_mac("test4")
for row in sol[0]:
    print(row)

Numarul total de celule negre: 86
([], 0)
[False, False, False, False, False, False, False, False, False, False, False, False, True, True, True]
[False, False, False, False, False, True, True, True, True, False, False, True, True, False, True]
[False, False, False, True, True, True, False, False, True, False, False, True, False, True, False]
[True, True, True, False, True, False, False, True, True, False, False, True, False, False, False]
[True, True, False, False, False, False, False, True, False, False, False, True, True, False, False]
[False, True, True, True, True, False, False, True, True, False, False, False, True, True, False]
[False, False, False, False, True, True, False, False, True, True, False, False, False, True, True]
[False, False, False, False, False, True, False, False, False, True, True, True, False, False, True]
[False, False, False, False, False, True, True, False, False, False, False, True, True, False, True]
[False, False, False, True, True, True, True, False, Fal

In concluzie, se poate vedea cu ochiul liber ca cea mai buna implementare este MAC din ambele puncte de vedere (spatial si temporal).
Acest lucru se datoreaza numarului mic de vecini pe care MAC ni-l ofera (deci vom explora mult mai putin comparativ cu celelalte variante)
